# Credit Decision Engine (Unified SHAP Architecture)
A production-grade decision layer with feature-level explainability across all model layers.

In [36]:
import pandas as pd
import numpy as np
import shap
import joblib
import json
import warnings
from collections import defaultdict

warnings.filterwarnings('ignore')

In [37]:

pd_model = joblib.load('pd_model.joblib')
pd_scaler = joblib.load('pd_scaler.joblib')
pd_features = joblib.load('pd_features.joblib')

iso_model = joblib.load('isolation_forest.joblib')
if_scaler = joblib.load('if_scaler.joblib')
if_features = ['avgMonthlyIncome', 'incomeCV', 'expenseRatio', 'emiRatio', 'avgMonthlyBalance', 'bounceCount']

risk_model = joblib.load('risk_random_forest.joblib')
risk_features = joblib.load('risk_features.joblib')

hybrid_model = joblib.load('hybrid_credit_score_model.joblib')
hybrid_features = joblib.load('hybrid_features.joblib')

q_table = joblib.load('q_learning_model.joblib')
q_bins = joblib.load('q_learning_bins.joblib')
q_features = joblib.load('q_learning_features.joblib')


bg_data = pd.read_csv('../data/synthetic/features_only.csv') # Background Data

X_pd_all = pd_scaler.transform(bg_data[pd_features])
bg_data['PD'] = pd_model.predict_proba(X_pd_all)[:, 1] #Store PD in bg_data

X_if_all = if_scaler.transform(bg_data[if_features])
if_scores = iso_model.decision_function(X_if_all) # Gets anomaly score
bg_data['anomalyFlag'] = (if_scores < -0.05).astype(int) # 1 = anomaly, 0 = normal

X_pd_bg = pd_scaler.transform(bg_data[pd_features])
X_if_bg = if_scaler.transform(bg_data[if_features])
X_risk_bg = bg_data[risk_features]
X_hybrid_bg = bg_data[hybrid_features]

bg_data.head()

,avgMonthlyIncome,incomeCV,expenseRatio,emiRatio,avgMonthlyBalance,bounceCount,accountAgeMonths,PD,anomalyFlag
0,315210.90,0.050,0.314,0.121,67185.96,0,100,0.003444,0
1,483386.74,0.038,0.292,0.125,210459.98,0,113,0.007820,0
2,488210.14,0.039,0.386,0.197,520559.06,1,105,0.059321,0
3,350279.88,0.040,0.408,0.129,365002.37,0,108,0.015605,0
4,449978.33,0.045,0.343,0.189,201947.71,0,90,0.011609,0


In [38]:
BUSINESS_MAPPING = {
    "avgMonthlyIncome": "Monthly Income Level",
    "incomeCV": "Income Volatility",
    "expenseRatio": "Monthly Expense Burden",
    "emiRatio": "Existing Debt Commitments (EMI)",
    "avgMonthlyBalance": "Liquidity Reserve",
    "bounceCount": "Historical Payment Bounces",
    "accountAgeMonths": "Banking Relationship Age",
    "PD": "Probability of Default Signal",
    "anomalyFlag": "Unusual Transaction Behavior",
    "anomaly": "Anomaly Signal Intensity",
    "HybridCreditScore": "Consolidated Credit Score"
}

### 🧠 SHAP Explainer Strategy

This document explains the SHAP explainer selection strategy for each model used in the credit risk architecture.

---

### 📌 Overview

| Model | Type | SHAP Explainer | Reason |
|-------|------|---------------|--------|
| PD Model | Logistic Regression | `LinearExplainer` | Exact + Fast for linear models |
| Isolation Forest | Tree-Based | `TreeExplainer` | Optimized for tree ensembles |
| Risk Random Forest | Tree Ensemble | `TreeExplainer` | Aggregates contributions across trees |
| Hybrid Credit Model | Mixed Pipeline | `shap.Explainer()` | Automatic backend detection |

---

In [39]:
pd_explainer = shap.LinearExplainer(pd_model, X_pd_bg)
if_explainer = shap.TreeExplainer(iso_model)
risk_explainer = shap.TreeExplainer(risk_model)
hybrid_explainer = shap.Explainer(hybrid_model, X_hybrid_bg)

### 🧠 Why RL Q-Table Requires `KernelExplainer`

- ❌ RL Policy (Q-table) is **NOT** a standard ML model  
- ❌ It has **no weights, no trees, no gradients**  
- ❌ It is **not parameterized**  
- ✅ It is simply a **lookup table (dictionary)**  

`KernelExplainer` :

- ✔ Works for any function
- ✔ No knowledge of internals needed

### Define Prototype of our RL Model And send to `KernelExplainer`


In [ ]:
#Function is Behaive Like Model
#Function Take Parameter as X => Produce Result(Array)

def q_policy_func(X):
    results = []

    for row in X:
        #Create tuple
        s = (np.digitize(row[0], q_bins['pd']),
             np.digitize(row[1], q_bins['anom']),
             np.digitize(row[2], q_bins['cs']))
        #Find Q Values From Q Table    
        q_vals = q_table.get(s, np.zeros(4))
        #Result As a Max
        results.append(np.max(q_vals))
    
    return np.array(results)

# KernelExplainer only needs a prototype
#So we can send Demo 3 Rows only
rl_bg = np.array([[0.1, 0.1, 600], [0.5, 0.5, 400], [0.1, 0.8, 400]])
rl_explainer = shap.KernelExplainer(q_policy_func, rl_bg)

In [41]:
def top_shap_features(values, names, k=3):
    idx = np.argsort(np.abs(values))[::-1][:k]
    return [f"{BUSINESS_MAPPING.get(names[i], names[i])} ({values[i]:+.3f})" for i in idx]


In [ ]:
def get_decision_explanation(input_row, explain=True):
    df_input = pd.DataFrame([input_row])
    response = {}
    
    # 1. PD Layer
    X_pd = pd_scaler.transform(df_input[pd_features])
    pd_val = pd_model.predict_proba(X_pd)[0, 1]
    response["PD"] = {"value": round(float(pd_val), 4)}
    
    if explain:
        pd_exp = pd_explainer(X_pd)
        pd_shap_vals = pd_exp.values.flatten()
        response["PD"]["top_factors"] = top_shap_features(pd_shap_vals, pd_features)

    # 2. Anomaly Layer
    X_if = if_scaler.transform(df_input[if_features])
    if_score = iso_model.decision_function(X_if)[0]
    response["Anomaly"] = {"score": round(float(if_score), 4)}
    
    if explain:
        if_exp = if_explainer(X_if)
        if_vals = if_exp.values.flatten()
        response["Anomaly"]["top_factors"] = top_shap_features(if_vals, if_features)

    # 3. Risk Layer
    df_risk = df_input.copy()
    df_risk['PD'] = pd_val
    df_risk['anomalyFlag'] = 1 if if_score < -0.05 else 0
    X_risk = df_risk[risk_features]
    risk_idx = int(risk_model.predict(X_risk)[0])
    risk_label = ["LOW", "MEDIUM", "HIGH"][risk_idx]
    response["RiskLabel"] = {"label": risk_label}
    
    if explain:
        risk_exp = risk_explainer(X_risk)
        if risk_exp.values.ndim == 3: 
            risk_vals = risk_exp.values[0, :, risk_idx]
        else:
            risk_vals = risk_exp.values.flatten()
        response["RiskLabel"]["drivers"] = top_shap_features(risk_vals, risk_features)

    # 4. Hybrid Score Layer
    X_hyb = df_input[hybrid_features]
    score_val = hybrid_model.predict(X_hyb)[0]
    response["HybridScore"] = {"value": round(float(score_val), 1)}
    
    if explain:
        hyb_exp = hybrid_explainer(X_hyb)
        hyb_vals = hyb_exp.values.flatten()
        response["HybridScore"]["factors"] = top_shap_features(hyb_vals, hybrid_features)

    # 5. RL Action Layer (Recommended Policy)
    anom_norm = np.clip(1.0 - (if_score + 0.5), 0, 1)
    X_rl = np.array([[pd_val, anom_norm, score_val]])
    s = (np.digitize(X_rl[0][0], q_bins['pd']),
         np.digitize(X_rl[0][1], q_bins['anom']),
         np.digitize(X_rl[0][2], q_bins['cs']))
    
    action_idx = int(np.argmax(q_table.get(s, np.zeros(4))))
    actions = ["REJECT", "APPROVE_LOW", "APPROVE_MEDIUM", "APPROVE_HIGH"]
    response["RL_Recommendation"] = {"action": actions[action_idx]}
    
    if explain:
        rl_exp = rl_explainer.shap_values(X_rl, silent=True)
        rl_vals = rl_exp.flatten()
        response["RL_Recommendation"]["rationales"] = top_shap_features(rl_vals, q_features)

    return response

print('✅ Architecture Phase 5, 6, & 7: Unified Pipeline ready.')

✅ Architecture Phase 5, 6, & 7: Unified Pipeline ready.


In [43]:
# Phase 9: Validate SHAP Sanity
elite_customer = {
    'avgMonthlyIncome': 150000, 'incomeCV': 0.02, 'expenseRatio': 0.15, 
    'emiRatio': 0.05, 'avgMonthlyBalance': 100000, 'bounceCount': 0, 'accountAgeMonths': 60
}

risky_customer = {
    'avgMonthlyIncome': 30000, 'incomeCV': 0.40, 'expenseRatio': 0.80, 
    'emiRatio': 0.60, 'avgMonthlyBalance': 500, 'bounceCount': 3, 'accountAgeMonths': 6
}

print('--- Elite Customer Explanation ---')
print(json.dumps(get_decision_explanation(elite_customer), indent=2))

print('\n--- Risky Customer Explanation ---')
print(json.dumps(get_decision_explanation(risky_customer), indent=2))

--- Elite Customer Explanation ---
{
  "PD": {
    "value": 0.0,
    "top_factors": [
      "Monthly Expense Burden (-5.444)",
      "Existing Debt Commitments (EMI) (-1.931)",
      "Historical Payment Bounces (-0.331)"
    ]
  },
  "Anomaly": {
    "score": 0.0955,
    "top_factors": [
      "Monthly Expense Burden (-1.447)",
      "Existing Debt Commitments (EMI) (-0.774)",
      "Liquidity Reserve (+0.350)"
    ]
  },
  "RiskLabel": {
    "label": "LOW",
    "drivers": [
      "Probability of Default Signal (+0.359)",
      "Monthly Expense Burden (+0.113)",
      "Existing Debt Commitments (EMI) (+0.093)"
    ]
  },
  "HybridScore": {
    "value": 606.3,
    "factors": [
      "Monthly Expense Burden (+30.253)",
      "Existing Debt Commitments (EMI) (+22.068)",
      "Historical Payment Bounces (+19.715)"
    ]
  },
  "RL_Recommendation": {
    "action": "REJECT",
    "rationales": [
      "Anomaly Signal Intensity (-33.915)",
      "Probability of Default Signal (+20.430)",
    